In [ ]:
pip install langchain

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
pip install --upgrade --quiet langchain-google-community[drive]

In [ ]:
from langchain.document_loaders import GoogleDriveLoader
import os
from langchain_google_community import GoogleDriveLoader
from langchain_community.document_loaders import UnstructuredFileIOLoader

In [ ]:
GOOGLE_APPLICATION_CREDENTIALS=""
from getpass import getpass
DEEPINFRA_API_TOKEN = getpass()

# Google Authentication

In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')


# Setting up the llm and taking in the google doc

In [ ]:
os.environ["DEEPINFRA_API_TOKEN"] = DEEPINFRA_API_TOKEN
from langchain_community.llms import DeepInfra
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]

llm = DeepInfra(model_id="meta-llama/Meta-Llama-3-70B-Instruct")
llm.model_kwargs = {
    "temperature": 0.5,
    "repetition_penalty": 1.2,
    "max_new_tokens": 250,
    "top_p": 0.9,
}
doc_id = "" #Put your Doc ID here
loader = GoogleDriveLoader(
    document_ids=[doc_id],
    credentials_path = "/content/credentials.json",
)


In [ ]:
docs=loader.load()

# Summarizes the document

In [ ]:
from langchain.chains.summarize import load_summarize_chain


In [ ]:
chain =load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(docs)

# You can ask whatever queries you want

In [ ]:
from langchain.chains.question_answering import load_qa_chain

In [ ]:
query="" #input your query here

In [ ]:
chain=load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)